<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Formatting" data-toc-modified-id="Formatting-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Formatting</a></span><ul class="toc-item"><li><span><a href="#animal-gut" data-toc-modified-id="animal-gut-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>animal-gut</a></span></li><li><span><a href="#TUK" data-toc-modified-id="TUK-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>TUK</a></span></li></ul></li><li><span><a href="#Merging" data-toc-modified-id="Merging-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Merging</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Writing-final-table" data-toc-modified-id="Writing-final-table-6.0.1"><span class="toc-item-num">6.0.1&nbsp;&nbsp;</span>Writing final table</a></span></li></ul></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Get sample accessions for real datasets used in this study
* https://docs.google.com/spreadsheets/d/1O6EHD8SVCd7yaZlUCJPj4okBRPdJJrYW6AtFJeUnXCs/edit#gid=0

# Var

In [44]:
work_dir = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/real_data_eval/accessions/'

samples_file = file.path(work_dir, 'samples.tsv')

animal_gut_file = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/ENA_upload/ENA_accessions.tsv'
tuk_file = '/ebio/abt3_projects/TwinsUK/vadinCA11/ENA_accessions.tsv'

# params
threads = 8

# Init

In [4]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(tidytable)
library(LeyLabRMisc)
Init(threads=threads)

In [10]:
clustermq_setup(scheduler = 'multicore')
make_dir(work_dir)

Directory already exists: /ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/real_data_eval/accessions/ 


# Load

In [12]:
samples = Fread(samples_file)
samples

Dataset,Study.identifier,Misassembly.rate
<chr>,<chr>,<dbl>
TwinsUK,99492,0.046
TwinsUK,94512,0.042
⋮,⋮,⋮
UHGG,ERP013562,0.037
UHGG,ERP020710,0.035


In [8]:
animal_gut = Fread(animal_gut_file)
animal_gut

Type,Accession,Accession2,Unique.Name
<chr>,<chr>,<chr>,<chr>
Sample,ERS4537005,SAMEA6809356,X101_Horse
Sample,ERS4537006,SAMEA6809357,X102_Kulan
⋮,⋮,⋮,⋮
Sample,ERS4537292,SAMEA6809643,XF44_Fallow_Deer
Sample,ERS4537293,SAMEA6809644,XF90_Domestic_Dog


In [45]:
tuk = Fread(tuk_file)
tuk

Accession,SampleID_ENA,SampleID_TUK.original,SampleID_TUK
<chr>,<chr>,<chr>,<int>
ERS5053140,VadinCA11_2_1171_p10__G12__209_42,1171,1002424
ERS5053139,VadinCA11_39_4632_S4_p2__D4_,4632,1002874
⋮,⋮,⋮,⋮
ERS5053012,VadinCA11_130_mock,mock,NA
ERS5053011,VadinCA11_129_water,water,NA


# Formatting

## animal-gut

In [49]:
samples_ag = samples %>%
    filter.(Dataset == 'Animal-gut')
overlap(samples_ag$Study.identifier, animal_gut$Unique.Name)

intersect(x,y): 5 
setdiff(x,y): 0 
setdiff(y,x): 284 
union(x,y): 289 


In [52]:
samples_ag = samples_ag %>%
    inner_join.(animal_gut %>% select.(Unique.Name, Accession), 
                c('Study.identifier' = 'Unique.Name'))
samples_ag

Dataset,Study.identifier,Misassembly.rate,Accession
<chr>,<chr>,<dbl>,<chr>
Animal-gut,X247_Common_Hamster,0.045,ERS4537104
Animal-gut,X332_Harbor_Porpoise,0.053,ERS4537154
⋮,⋮,⋮,⋮
Animal-gut,X415_White_Rhinoceros,0.059,ERS4537227
Animal-gut,X85_Domestic_Dog,0.018,ERS4537271


## TUK

In [53]:
samples_tuk = samples %>%
    filter.(Dataset == 'TwinsUK')
overlap(samples_tuk$Study.identifier, tuk$SampleID_TUK.original)

intersect(x,y): 98 
setdiff(x,y): 0 
setdiff(y,x): 32 
union(x,y): 130 


In [56]:
samples_tuk = samples_tuk %>%
    inner_join.(tuk %>% select.(SampleID_TUK.original, Accession), 
                c('Study.identifier' = 'SampleID_TUK.original'))
samples_tuk

Dataset,Study.identifier,Misassembly.rate,Accession
<chr>,<chr>,<dbl>,<chr>
TwinsUK,8571,0.023,ERS5053138
TwinsUK,9311,0.028,ERS5053137
⋮,⋮,⋮,⋮
TwinsUK,1020132,0.025,ERS5053015
TwinsUK,1020174,0.038,ERS5053013


# Merging

In [63]:
samples_uhgg = samples %>%
    filter.(Dataset == 'UHGG') %>%
    mutate.(Accession = Study.identifier) 
samples_uhgg

Dataset,Study.identifier,Misassembly.rate,Accession
<chr>,<chr>,<dbl>,<chr>
UHGG,ERP006678,0.168,ERP006678
UHGG,ERP003612,0.132,ERP003612
⋮,⋮,⋮,⋮
UHGG,ERP013562,0.037,ERP013562
UHGG,ERP020710,0.035,ERP020710


In [64]:
accs = rbindlist(list(
    samples_uhgg, samples_tuk, samples_ag
))
accs

Dataset,Study.identifier,Misassembly.rate,Accession
<chr>,<chr>,<dbl>,<chr>
UHGG,ERP006678,0.168,ERP006678
UHGG,ERP003612,0.132,ERP003612
⋮,⋮,⋮,⋮
Animal-gut,X415_White_Rhinoceros,0.059,ERS4537227
Animal-gut,X85_Domestic_Dog,0.018,ERS4537271


### Writing final table

In [67]:
# writing final table
F = file.path(work_dir, 'accessions.tsv')
write_table(accs, F)

File written: /ebio/abt3_projects/databases_no-backup/DeepMAsED/version2/real_data_eval/accessions//accessions.tsv 


# sessionInfo

In [68]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.6 LTS

Matrix products: default
BLAS/LAPACK: /tmp/global2/nyoungblut/code/general/conda_envs/general-tidyverse/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] clustermq_0.8.95.2 LeyLabRMisc_0.2.1  tidytable_0.6.6    data.table_1.14.2 
[5] ggplot2_3.3.5      tidyr_1.1.4        dplyr_1.0.7       

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.7       plyr_1.8.6       pillar_1.6.4     compiler_4.1.1  
 [5] base64enc_0